In [312]:
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm

df_pred = pd.read_csv(r'W:\PycharmProjects\kaggle-RSNA\input\train_predictions.csv')
df_pred.head()

,StudyInstanceUID,Slice,ImageHeight,ImageWidth,SliceThickness,ImagePositionPatient_x,ImagePositionPatient_y,ImagePositionPatient_z,C1,C2,...,C5_effnet_frac,C6_effnet_frac,C7_effnet_frac,C1_effnet_vert,C2_effnet_vert,C3_effnet_vert,C4_effnet_vert,C5_effnet_vert,C6_effnet_vert,C7_effnet_vert
0,1.2.826.0.1.3680043.10032,1,512,512,0.625,-82.0,11.712,37.166,0,0,...,0.000693,0.000422,0.000429,0.001040,0.001831,0.003172,0.007347,0.001095,0.000229,0.000194
1,1.2.826.0.1.3680043.10032,2,512,512,0.625,-82.0,11.712,36.541,0,0,...,0.000600,0.000361,0.000354,0.000922,0.001411,0.002350,0.005200,0.000752,0.000163,0.000147
2,1.2.826.0.1.3680043.10032,3,512,512,0.625,-82.0,11.712,35.916,0,0,...,0.000624,0.000379,0.000424,0.000915,0.001315,0.002192,0.004757,0.000789,0.000190,0.000194
3,1.2.826.0.1.3680043.10032,4,512,512,0.625,-82.0,11.712,35.291,0,0,...,0.000638,0.000382,0.000439,0.001129,0.002209,0.002834,0.005120,0.000680,0.000173,0.000203
4,1.2.826.0.1.3680043.10032,5,512,512,0.625,-82.0,11.712,34.666,0,0,...,0.000707,0.000386,0.000494,0.000925,0.001265,0.002405,0.005688,0.000738,0.000155,0.000220


In [313]:
df_y = pd.read_csv(f'../input/rsna-2022-cervical-spine-fracture-detection/train.csv')
df_y = df_y.query('StudyInstanceUID != "1.2.826.0.1.3680043.20574"').reset_index(drop=True)
df_y = df_y[1400:]
id_lst = df_y['StudyInstanceUID']
df_y.head(3)

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0


In [314]:
df_pred = df_pred[df_pred["StudyInstanceUID"].isin(id_lst)]
df_pred

,StudyInstanceUID,Slice,ImageHeight,ImageWidth,SliceThickness,ImagePositionPatient_x,ImagePositionPatient_y,ImagePositionPatient_z,C1,C2,...,C5_effnet_frac,C6_effnet_frac,C7_effnet_frac,C1_effnet_vert,C2_effnet_vert,C3_effnet_vert,C4_effnet_vert,C5_effnet_vert,C6_effnet_vert,C7_effnet_vert
0,1.2.826.0.1.3680043.10032,1,512,512,0.625,-82.000,11.7120,37.166,0,0,...,0.000693,0.000422,0.000429,0.001040,0.001831,0.003172,0.007347,0.001095,0.000229,0.000194
1,1.2.826.0.1.3680043.10032,2,512,512,0.625,-82.000,11.7120,36.541,0,0,...,0.000600,0.000361,0.000354,0.000922,0.001411,0.002350,0.005200,0.000752,0.000163,0.000147
2,1.2.826.0.1.3680043.10032,3,512,512,0.625,-82.000,11.7120,35.916,0,0,...,0.000624,0.000379,0.000424,0.000915,0.001315,0.002192,0.004757,0.000789,0.000190,0.000194
3,1.2.826.0.1.3680043.10032,4,512,512,0.625,-82.000,11.7120,35.291,0,0,...,0.000638,0.000382,0.000439,0.001129,0.002209,0.002834,0.005120,0.000680,0.000173,0.000203
4,1.2.826.0.1.3680043.10032,5,512,512,0.625,-82.000,11.7120,34.666,0,0,...,0.000707,0.000386,0.000494,0.000925,0.001265,0.002405,0.005688,0.000738,0.000155,0.000220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711498,1.2.826.0.1.3680043.9994,260,512,512,1.000,-130.412,-119.7876,-690.700,0,0,...,0.000709,0.000575,0.002502,0.001341,0.000577,0.000918,0.000582,0.000162,0.000181,0.000951
711499,1.2.826.0.1.3680043.9994,261,512,512,1.000,-130.412,-119.7876,-691.500,0,0,...,0.000631,0.000542,0.002846,0.000752,0.000553,0.001124,0.000634,0.000145,0.000185,0.001394
711500,1.2.826.0.1.3680043.9994,262,512,512,1.000,-130.412,-119.7876,-692.300,0,0,...,0.000735,0.000677,0.004280,0.000548,0.000315,0.001044,0.000856,0.000190,0.000259,0.002672
711501,1.2.826.0.1.3680043.9994,263,512,512,1.000,-130.412,-119.7876,-693.100,0,0,...,0.000911,0.000770,0.004383,0.000577,0.000278,0.001192,0.001138,0.000257,0.000285,0.002571


In [315]:
DEVICE = 'cpu'


In [316]:
target_cols = ['patient_overall'] + [f'C{i}_fracture' for i in range(1, 8)]
frac_cols = [f'C{i}_effnet_frac' for i in range(1, 8)]
vert_cols = [f'C{i}_effnet_vert' for i in range(1, 8)]
def patient_prediction(df):
    c1c7 = np.average(df[frac_cols].values, axis=0, weights=df[vert_cols].values)
    pred_patient_overall = 1 - np.prod(1 - c1c7)
    return np.concatenate([[pred_patient_overall], c1c7])

df_patient_pred = df_pred.groupby('StudyInstanceUID').apply(lambda df: patient_prediction(df)).to_frame('pred').join(df_pred.groupby('StudyInstanceUID')[target_cols].mean())

In [317]:
predictions = np.stack(df_patient_pred.pred.values.tolist())
predictions.dtype

dtype('float64')

In [318]:
targets = df_patient_pred[target_cols].values
targets.dtype

dtype('float64')

In [319]:
def weighted_loss(y_pred_logit, y, reduction='mean', verbose=False):
    """
    Weighted loss
    We reuse torch.nn.functional.binary_cross_entropy_with_logits here. pos_weight and weights combined give us necessary coefficients described in https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/discussion/340392

    See also this explanation: https://www.kaggle.com/code/samuelcortinhas/rsna-fracture-detection-in-depth-eda/notebook
    """

    neg_weights = (torch.tensor([7., 1, 1, 1, 1, 1, 1, 1]) if y_pred_logit.shape[-1] == 8 else torch.ones(y_pred_logit.shape[-1])).to(DEVICE)
    pos_weights = (torch.tensor([14., 2, 2, 2, 2, 2, 2, 2]) if y_pred_logit.shape[-1] == 8 else torch.ones(y_pred_logit.shape[-1]) * 2.).to(DEVICE)

    loss = torch.nn.functional.binary_cross_entropy_with_logits(
        y_pred_logit,
        y,
        reduction='none',
    )

    if verbose:
        print('loss', loss)

    pos_weights = y * pos_weights.unsqueeze(0)
    neg_weights = (1 - y) * neg_weights.unsqueeze(0)
    all_weights = pos_weights + neg_weights

    if verbose:
        print('all weights', all_weights)

    loss *= all_weights
    if verbose:
        print('weighted loss', loss)

    norm = torch.sum(all_weights, dim=1).unsqueeze(1)
    if verbose:
        print('normalization factors', norm)

    loss /= norm
    if verbose:
        print('normalized loss', loss)

    loss = torch.sum(loss, dim=1)
    if verbose:
        print('summed up over patient_overall-C1-C7 loss', loss)

    if reduction == 'mean':
        return torch.mean(loss)
    return loss

In [320]:
DEVICE = 'cpu'
print('CV score:', weighted_loss(torch.logit(torch.as_tensor(predictions)).float().to(DEVICE), torch.as_tensor(targets).float().to(DEVICE)))

CV score: tensor(0.4955)


In [321]:
UID_target_dict = {}
c1c7 = [f'C{i}' for i in range(1, 8)]

for i in range(7):
    for row in df_y.iterrows():
        UID_target_dict[row[1][0]+'_'+c1c7[i]] = row[1][i+2]
len(UID_target_dict)

14126

In [322]:
UID_x_dict = {}
vert_c1c7 = [f'C{i}_effnet_vert' for i in range(1, 8)]
frac_c1c7 = [f'C{i}_effnet_frac' for i in range(1, 8)]
for UID_C in tqdm(UID_target_dict.keys()):
    UID = UID_C.split('_C')[0]
    cn = int(UID_C.split('_C')[1])
    df1 = df_pred.loc[df_pred['StudyInstanceUID']==UID].copy()
    df1[vert_c1c7] = (df1[vert_c1c7] > 0.5).astype(int)

    x = np.average(df1.loc[df1[f'C{cn}_effnet_vert']>0.5][frac_c1c7].values,axis=0).tolist()
    UID_x_dict[UID_C] =  x
len(UID_x_dict)

  9%|▉         | 1340/14126 [00:40<06:50, 31.18it/s]W:\Anaconda3\envs\rsna39\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
W:\Anaconda3\envs\rsna39\lib\site-packages\numpy\core\_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
 30%|██▉       | 4183/14126 [02:06<05:00, 33.04it/s]W:\Anaconda3\envs\rsna39\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
W:\Anaconda3\envs\rsna39\lib\site-packages\numpy\core\_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
 35%|███▌      | 4955/14126 [02:29<04:52, 31.35it/s]W:\Anaconda3\envs\rsna39\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
W:\Anaconda3\envs\rsna39\lib\site-packages\numpy\core\_methods.py:182: RuntimeWarning: invalid value encou

14126

In [323]:
y = np.array([[]]).T
x = np.array([[]]*7).T
for UID in UID_target_dict.keys():
    x = np.vstack((x,np.array(UID_x_dict[UID]).T))
    y = np.vstack((y,np.array(UID_target_dict[UID]).T))
y = y.ravel()
x.shape

(14126, 7)

In [324]:
x = np.nan_to_num(x)
x = np.maximum(x, 0.000001)
x = np.minimum(x,0.999999)

In [325]:
import joblib
from sklearn.ensemble import RandomForestClassifier
clf =  joblib.load('gb.model')

In [326]:
score_c = clf.score(x, y)
score_c

0.9118646467506725

In [327]:
from sklearn.metrics import roc_curve, auc
y_test_proba_rfc = clf.predict_proba(x)
false_positive_rate_rfc, recall_rfc, thresholds_rfc = roc_curve(y, y_test_proba_rfc[:, 1])
auc(false_positive_rate_rfc, recall_rfc)

0.8145755699962118

In [328]:
rf_pred = clf.predict_proba(x)
rf_pred[:,1]

array([0.9301187 , 0.03681638, 0.03056765, ..., 0.2573939 , 0.2940773 ,
       0.15282642])

In [329]:
UID_ypred_dict = {}
i=0
for UID_C in tqdm(UID_target_dict.keys()):
    UID = UID_C.split('_C')[0]
    cn = int(UID_C.split('_C')[1])
    if UID in UID_ypred_dict.keys():
        UID_ypred_dict[UID][cn-1] = rf_pred[i,1]
    else:
        UID_ypred_dict[UID] = [0.,0.,0.,0.,0.,0.,0.]
        UID_ypred_dict[UID][cn-1] = rf_pred[i,1]
    i+=1
UID_ypred_dict.__len__()

100%|██████████| 14126/14126 [00:00<00:00, 806884.72it/s]


2018

In [330]:
clf2 =  joblib.load('rf2.model')


In [331]:
for i in tqdm(UID_ypred_dict.items()):
    c1c7 = (np.array(i[1]))

    pred_patient_overall1 = 1 - np.prod(1 - c1c7)
    # pred_patient_overall2 = clf2.predict_proba(np.array(i[1]).reshape(1, -1))[0][1]

    # pred_patient_overall = (pred_patient_overall1 + pred_patient_overall2)/2
    UID_ypred_dict[i[0]].insert(0,pred_patient_overall1)
UID_ypred_dict['1.2.826.0.1.3680043.2813']

100%|██████████| 2018/2018 [00:00<00:00, 174885.44it/s]


[0.853706170619242,
 0.35611460388580946,
 0.49579854824266384,
 0.12523814264839148,
 0.1349580285560267,
 0.14387180936973745,
 0.15496645006614462,
 0.1768602513059201]

In [332]:
from sklearn.metrics import precision_score,accuracy_score


In [334]:
pred = np.array([[]]*8).T
target = np.array([[]]*8).T
ori_overall = np.array([[]]).T

for row in df_y.iterrows():
    ori_overall = np.vstack((ori_overall,np.array(1 if UID_ypred_dict[row[1][0]][0]>0.5 else 0)))

    pred = np.vstack((pred,UID_ypred_dict[row[1][0]]))
    target = np.vstack((target,[row[1][i] for i in range(1,9)]))
pred.shape

(2018, 8)

In [335]:
pred.astype(dtype=float)
pred = np.maximum(pred, 0.000001)
pred = np.minimum(pred,0.999999)
len(pred)

2018

In [336]:
target_overall = target[:,1]
ori_overall = ori_overall.ravel()
accuracy_score(ori_overall, target_overall) # overall的准确率

0.6015857284440039

In [337]:
DEVICE = 'cpu'
print('过rf之后CV score:', weighted_loss(torch.logit(torch.as_tensor(pred)).float().to(DEVICE), torch.as_tensor(target).float().to(DEVICE)))

过rf之后CV score: tensor(0.4488)
